In [1]:
!pip install torch>=2.0.0
!pip install diffusers>=0.24.0
!pip install transformers>=4.36.0
!pip install accelerate>=0.27.0
!pip install pillow>=10.0.0 

In [5]:
!pip install datasets

  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-19.0.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached aiohttp-3.11.16-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached multidict-6.4.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.3 kB)
  Using cached propcache-0.3.1-cp310-

In [6]:
import torch
from diffusers import DiffusionPipeline
from datasets import load_dataset
import os
from tqdm import tqdm
import json
from datetime import datetime


In [7]:
def setup_output_directory(base_dir="coco_generated"):
    """
    Create a timestamped output directory for the generated images
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = os.path.join(base_dir, f"run_{timestamp}")
    os.makedirs(output_dir, exist_ok=True)
    return output_dir


In [8]:
def initialize_model():
    """
    Initialize the DALL-E model
    """
    pipeline = DiffusionPipeline.from_pretrained(
        "dataautogpt3/ProteusV0.2",
        torch_dtype=torch.float16
    )
    
    if torch.cuda.is_available():
        pipeline = pipeline.to("cuda")
    
    return pipeline


In [9]:
def generate_image(pipeline, prompt, output_path, index):
    """
    Generate an image from a caption and save it
    """
    try:
        # Generate the image
        image = pipeline(prompt).images[0]
        
        # Create filename with index and first few words of caption
        safe_prompt = "".join(x for x in prompt[:30] if x.isalnum() or x in (' ', '-', '_')).strip()
        filename = f"image_{index:04d}_{safe_prompt}.png"
        full_path = os.path.join(output_path, filename)
        
        # Save the image
        image.save(full_path)
        
        # Return metadata
        return {
            "index": index,
            "caption": prompt,
            "filename": filename,
            "status": "success"
        }
    
    except Exception as e:
        return {
            "index": index,
            "caption": prompt,
            "error": str(e),
            "status": "failed"
        }


In [10]:
# Setup
output_dir = setup_output_directory()
pipeline = initialize_model()


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
unet/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /home/.cache/huggingface/hub/models--dataautogpt3--ProteusV0.2/snapshots/4437fb7de964a3b4f323aaa73d1c4c73d064b0cb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /home/.cache/huggingface/hub/models--dataautogpt3--ProteusV0.2/snapshots/4437fb7de964a3b4f323aaa73d1c4c73d064b0cb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /home/.cache/huggingface/hub/models--dataautogpt3--ProteusV0.2/snapshots/4437fb7de964a3b4f323aaa73d1c4c73d064b0cb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /home/.cache/huggingface/hub/models--dataautogpt3--ProteusV0.2/snapshots/4437fb7de964a3b4f323aaa73d1c4c73d064b0cb/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


In [11]:
print("Loading COCO dataset...")
dataset = load_dataset("lmms-lab/COCO-Caption2017", split="val[:100]")


Loading COCO dataset...


In [12]:
# Initialize metadata storage
metadata = []

# Process each caption
print("Generating images...")
for idx, item in enumerate(tqdm(dataset)):
    caption = item['answer'][0]  # Get the first caption
    result = generate_image(pipeline, caption, output_dir, idx)
    metadata.append(result)
    
    # Print status
    if result["status"] == "success":
        print(f"Generated image {idx}: {caption[:50]}...")
    else:
        print(f"Failed to generate image {idx}: {result['error']}")


Generating images...


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  1%|          | 1/100 [00:28<46:36, 28.24s/it]

Generated image 0: A black Honda motorcycle parked in front of a gara...


...

 99%|█████████▉| 99/100 [47:58<00:29, 29.08s/it]

Generated image 98: A bike is perched on a concrete girder on the road...


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 100/100 [48:27<00:00, 29.07s/it]

Generated image 99: A pretty young girl standing next to another prett...


In [13]:
# Save metadata
metadata_path = os.path.join(output_dir, "metadata.json")
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\nProcessing complete!")
print(f"Generated images saved in: {output_dir}")
print(f"Metadata saved in: {metadata_path}")



Processing complete!
Generated images saved in: coco_generated/run_20250413_154423
Metadata saved in: coco_generated/run_20250413_154423/metadata.json
